In [11]:
from scipy import spatial
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import random
import time
import math
tokenizer = RegexpTokenizer(r'\w+')


## Now with our data

In [1]:
import pandas as pd
import numpy as np
import random

### Remove when our part works
#data = pd.read_csv('../data/quora_duplicate_questions.tsv', sep='\t')

ItemsDF = pd.read_excel('Items.xlsx') #header=None, skiprows=1)
leftItems  = ItemsDF[['VariableId','ItemId','Text','SourceId','PoolID']]
rightItems = ItemsDF[['VariableId','ItemId','Text','SourceId','PoolID']]

train = []
test = []

train1 = 0
test1 = 0
trainsp0 = 0
train0 = 0
test0 = 0

random.seed(1)
for key, value in enumerate(leftItems.values):
    for key2, value2 in enumerate(rightItems.values):
        if value[1] < value2[1]:  #Only working with lower left triangle
            if value[0] == value2[0]:  # Same variable (our 1s)  50,212 pairs
                train.append([value[0]* value2[0],value[1],value2[1], value[2],value2[2],1])
                train1 +=1
            elif value[4] == value2[4]: #Same pool   #192,621 pairs
                if random.randint(1,4) == 1:  #Downsampling test set 1s           
                    test.append([value[0]* value2[0],value[1],value2[1], value[2],value2[2],1])
                    test1+=1
            elif value[3] == value2[3]: #Same paper 292,913 pairs
                if random.randint(1,6) == 1:  #Downsampling same paper zeros class               
                    train.append([value[0]* value2[0],value[1],value2[1], value[2],value2[2],0])
                    trainsp0 +=1
            else:  # Unrelated     158,552,457 pairs
                if random.randint(1,3158) == 1:  #Picking zeros for train set             
                    train.append([value[0]* value2[0],value[1],value2[1], value[2],value2[2],0])
                    train0+=1
                elif random.randint(1,3158) == 1:  #Picking zeros for testset        
                    test.append([value[0]* value2[0],value[1],value2[1], value[2],value2[2],0])
                    test0+=1

print('train1',train1)
print('test1',test1)
print('trainsp0',trainsp0)
print('train0',train0)
print('test0',test0)


                    
train = pd.DataFrame(train)
train = train.rename(columns = {0:'id', 1:'qid1',2:'quid2',3:'question1', 4:'question2',5:'is_duplicate'})

test = pd.DataFrame(test)
test = test.rename(columns = {0:'id', 1:'qid1',2:'quid2',3:'question1', 4:'question2',5:'is_duplicate'})


train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

train.to_csv('../data/train.csv', sep='\t')
test.to_csv('../data/test.csv', sep='\t')

train1 50212
test1 48296
trainsp0 48734
train0 50257
test0 50336


In [3]:
len(dfd)

17838

## Create a new column in your existing dataframe

## Import the GoogleWordVec

In [13]:
import pandas as pd
import csv

import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [14]:
word_vectors = model.wv

/Users/megoconnell/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


## Create an overlapping dictionary

In [15]:
import re
all_words = [word for sentence in ItemsDF['Text'] for word in re.split('\W', sentence)]
unique = (set(all_words))

In [16]:
a = unique
c = []
for word in a:
    if word in word_vectors.vocab:
        c.append(word)


In [17]:
dict1 = {}
for word in c:
    vword = model[word]
    vector = {word : vword }
    dict1.update(vector)

In [18]:
all_words1 = [word.lower() for sentence in ItemsDF['Text'] for word in re.split('\W', sentence)]
lower = (set(all_words))

In [19]:
Embeddings1 = dict1

In [20]:
stop_words = set(stopwords.words('english'))

## Functions!

In [21]:
def calculateCosine1(W1,W2):
    #resultww = []
    result = 1 - spatial.distance.cosine(Embeddings1[W1], Embeddings1[W2])
    #if W1.lower() in weightdict:
        
    result = round(result, 2)
    return(result)

In [22]:
def calculateSentenceSim1(S1,S2):
    pairs = []
    for index1,word1 in enumerate(S1):
        for index2,word2 in enumerate(S2):
            pairs.append([index1+1, word1,index2+1,word2,calculateCosine1(word1,word2)])
    pairs = sorted(pairs, key=lambda x:x[4], reverse=True)
    
    # Now create the similarity vector
    tempPairsVector = []
    for index, row in enumerate(pairs):


        foundOne=False
        for tempRow in tempPairsVector:

            if tempRow[0] == row[0] or tempRow[2] == row[2]:
                foundOne=True
        if foundOne==False:
            tempPairsVector.append(row)

#Now create the vector of the up to 20 highest cosines
    sentenceSimVector = [] #[None] * 20
    for index,row in enumerate(tempPairsVector):
        sentenceSimVector.append(row[4])
    
#Pad the similarity vector with nulls    
    length = 20 - len(sentenceSimVector)
    for x in range(0, length):
        sentenceSimVector.append(None)            
    return(sentenceSimVector)

## Now with all the data

## Set up the data

## Make a dictionary from the DataFrame

In [93]:
len(word_vectors.vocab.keys())

3000000

In [59]:
traintest = train.head(20)
traintest.head(1)

,id,qid1,quid2,question1,question2,is_duplicate
0,2558869596,28852,67266,I believe that it is my personal duty as CAPS'...,"To accomplish their work, employees are willin...",0


## Train

In [94]:
import re
new = []
for index, row in train.iterrows():
        itemslist1 = [row['question1']]
        itemslist2 = [row['question2']]
        #print(itemslist1)
        #print(itemslist2)
        words1 = ([word for sentence in itemslist1 for word in tokenizer.tokenize(sentence)])
        words2 = ([word for sentence in itemslist2 for word in tokenizer.tokenize(sentence)])
        #print(words1,words2)
        filtered_sentence1 = [w for w in words1 if not w in stop_words]
        filtered_sentence2 = [w for w in words2 if not w in stop_words]
        #print(filtered_sentence1,filtered_sentence2)
        for word in filtered_sentence1:
            if word not in Embeddings1.keys():
                filtered_sentence1.remove(word)
        for word in filtered_sentence1:
            if word not in Embeddings1.keys():
                filtered_sentence1.remove(word)
        for word in filtered_sentence2:
            if word not in Embeddings1.keys():
                filtered_sentence2.remove(word)
        for word in filtered_sentence2:
            if word not in Embeddings1.keys():
                filtered_sentence2.remove(word)     
        new.append([row['id'],row['qid1'], row['quid2'],row['is_duplicate'],calculateSentenceSim1(filtered_sentence1,filtered_sentence2)])       

## Test

In [100]:
import re
newt = []
for index, row in test.iterrows():
        itemslist1 = [row['question1']]
        itemslist2 = [row['question2']]
        #print(itemslist1)
        #print(itemslist2)
        words1 = ([word for sentence in itemslist1 for word in tokenizer.tokenize(sentence)])
        words2 = ([word for sentence in itemslist2 for word in tokenizer.tokenize(sentence)])
        #print(words1,words2)
        filtered_sentence1 = [w for w in words1 if not w in stop_words]
        filtered_sentence2 = [w for w in words2 if not w in stop_words]
        #print(filtered_sentence1,filtered_sentence2)
        for word in filtered_sentence1:
            if word not in Embeddings1.keys():
                filtered_sentence1.remove(word)
        for word in filtered_sentence1:
            if word not in Embeddings1.keys():
                filtered_sentence1.remove(word)
        for word in filtered_sentence2:
            if word not in Embeddings1.keys():
                filtered_sentence2.remove(word)
        for word in filtered_sentence2:
            if word not in Embeddings1.keys():
                filtered_sentence2.remove(word)     
        newt.append([row['id'],row['qid1'], row['quid2'],row['is_duplicate'],calculateSentenceSim1(filtered_sentence1,filtered_sentence2)])       

### 50212 trues

In [95]:
list1 = []
list1.append([])
list1 = []
list1.append([])
z = 0
for x in new:
    list2 = []
    for y in x[4]:
        
        #print(y)
        #print(x[0],x[1],x[2], x[3],y)
        list2.append(y)
        #for m in list2:
            #list3.append(m)
            #list3[z].extend(m)
    list1[z].append(x[0])
    list1[z].append(x[1])
    list1[z].append(x[2])
    list1[z].append(x[3])
    list1[z].extend(list2)
    list1[z]
    z += 1
    list1.append([])
list1[0]
#list3

[2558869596,
 28852,
 67266,
 0,
 1.0,
 0.38,
 0.34,
 0.28,
 0.25,
 0.25,
 0.2,
 0.14,
 0.14,
 0.13,
 0.12,
 0.07,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [102]:
listT1 = []
listT1.append([])
listT1 = []
listT1.append([])
z = 0
for x in newt:
    list2 = []
    for y in x[4]:
        
        #print(y)
        #print(x[0],x[1],x[2], x[3],y)
        list2.append(y)
        #for m in list2:
            #list3.append(m)
            #list3[z].extend(m)
    listT1[z].append(x[0])
    listT1[z].append(x[1])
    listT1[z].append(x[2])
    listT1[z].append(x[3])
    listT1[z].extend(list2)
    listT1[z]
    z += 1
    listT1.append([])
listT1[0]
#list3

[6649157254,
 60393,
 60836,
 0,
 1.0,
 1.0,
 0.68,
 0.18,
 0.16,
 0.14,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [103]:
tableDF2 = pd.DataFrame(list1)
tableDF2T = pd.DataFrame(listT1)

In [104]:
tableDF2 = tableDF2.rename(columns = {0:'id', 1:'quid1',2:'quid2',3:'Target'})
tableDF2T = tableDF2T.rename(columns = {0:'id', 1:'quid1',2:'quid2',3:'Target'})

## Downsample

In [105]:
GoogleNormal = tableDF2.iloc[:,0:24]
GoogleNormalT = tableDF2T.iloc[:,0:24]

In [106]:
GoogleNormal.to_csv('GooglePoolTrain.csv')
GoogleNormalT.to_csv('GooglePoolTest.csv')